## Introduction

In this assignment, you are going to build a classifier for named entities from the Groningen Meaning Bank corpus.  Named entity recognition (NER) takes noun phrases from a text and identifies whether they are persons, organizations, and so on.  You will be using the Groningen Meaning Bank named entity corpus available on mltgpu at `/scratch/lt2222-v21-resources/GMB_dataset.txt`.  In this version of the task, you will assume we know *that* something is a named entity, and instead use multi-class classification to identify its type.  So you will be doing named entity classification but *not* recognition.

The data looks like this: 

```
3996    182.0   Nicole  NNP     B-per
3997    182.0   Ritchie NNP     I-per
3998    182.0   is      VBZ     O
3999    182.0   pregnant        JJ      O
4000    182.0   .       .       O
4001    183.0   Speaking        VBG     O
4002    183.0   to      TO      O
4003    183.0   ABC     NNP     B-org
4004    183.0   News    NNP     I-org
4005    183.0   interviewer     NN      O
4006    183.0   Dianne  NNP     B-per
4007    183.0   Sawyer  NNP     I-per
4008    183.0   ,       ,       O
4009    183.0   the     DT      O
4010    183.0   25-year-old     JJ      O
4011    183.0   co-star NN      O
4012    183.0   of      IN      O
4013    183.0   TV      NN      O
4014    183.0   's      POS     O
4015    183.0   The     DT      B-art
4016    183.0   Simple  NNP     I-art
4017    183.0   Life    NNP     I-art
4018    183.0   said    VBD     O
4019    183.0   she     PRP     O
4020    183.0   is      VBZ     O
4021    183.0   almost  RB      O
4022    183.0   four    CD      O
4023    183.0   months  NNS     O
4024    183.0   along   IN      O
4025    183.0   in      IN      O
4026    183.0   her     PRP$    O
4027    183.0   pregnancy       NN      O
4028    183.0   .       .       O
```

The first column is the line number.  The second column is a sentence number (for some reason given as a float; ignore it).  The third column is the word.  The fourth column is a part of speech (POS) tag in Penn Treebank format.  The last column contains the named entity annotation. 

The annotation works like this.  Every `O` just means that the row does not represent a named entity.  `B-xyx` means the first word in a named entity with type `xyx`. `I-xyz` means the second and later words of an `xyz` entity, if there are any.  That means that every time there's a `B` or an `I`, there's a named entity.  

The entity types in the corpus are `art`,
`eve`,
`geo`,
`gpe`,
`nat`,
`org`,
`per`,
and `tim`

Your task is the following.
1. To preprocess the text (lowercase and lemmatize; punctuation can be preserved as it gets its own rows).
2. To create instances from every from every identified named entity in the text with the type of the NE as the class, and a surrounding context of five words on either side as the features.  
3. To generate vectors and split the instances into training and testing datasets at random.
4. To train a support vector machine (via `sklearn.svm.LinearSVC`) for classifying the NERs.
5. To evaluate the performance of the classifier.

You will do this by modifying a separate file containing functions that will be called from this notebook as a module.  You can modify this notebook for testing purposes but please only submit the original.  You will document everything in Markdown in README.md and submit a GitHub repository URL.

This assignment is due on **Tuesday, 2021 March 9 at 23:59**.  It has **25 points** and **7 bonus points**.

In [19]:
import a2
from sklearn.svm import LinearSVC

In [20]:
gmbfile = open('/scratch/lt2222-v21-resources/GMB_dataset.txt', "r")

## Part 1 - preprocessing (3 points)

See step 1 above.  The data is coming to you as an unused file handle object.  You can return the data in any indexable form you like.  You can also choose to remove infrequent or uninformative words to reduce the size of the feature space. (Document this in README.md.)

In [21]:
inputdata = a2.preprocess(gmbfile)
gmbfile.close()
inputdata[20:40]

,Sentence #,Word,POS,Tag
20,1.0,from,IN,O
21,1.0,that,DT,O
22,1.0,country,NN,O
23,2.0,families,NNS,O
24,2.0,of,IN,O
25,2.0,soldier,NNS,O
26,2.0,kill,VBN,O
27,2.0,in,IN,O
28,2.0,the,DT,O
29,2.0,conflict,NN,O


## Part 2 - Creating instances (7 points)

Do step 2 above.  You will create a collection of Instance objects.  Remember to consider the case where the NE is at the beginning of a sentence or at the end, or close to either (you can create a special start token for that).  You can also start counting from before the `B` end of the NE mention and after the last `I` of the NE mention. That means that the instances should include things before and after the named entity mention, but not the named entity text itself.

In [22]:
instances = a2.create_instances(inputdata)

In [23]:
instances[20:40]

[Class: gpe Features: ['<S1>', '<S2>', '<S3>', '<S4>', '<S5>', 'this', 'week', 'restart', 'part', 'of'],
 Class: geo Features: ['the', 'conversion', 'process', 'at', 'its', 'nuclear', 'plant', '<E5>', '<E4>', '<E3>'],
 Class: gpe Features: ['<S1>', '<S2>', '<S3>', '<S4>', '<S5>', 'official', 'say', 'they', 'expect', 'to'],
 Class: tim Features: ['sensitive', 'part', 'of', 'the', 'plant', 'after', 'an', 'iaea', 'surveillance', 'system'],
 Class: org Features: ['the', 'plant', 'wednesday', 'after', 'an', 'surveillance', 'system', 'begin', 'function', '<E5>'],
 Class: org Features: ['<S2>', '<S3>', '<S4>', '<S5>', 'the', 'with', 'backing', 'have', 'threaten', 'to'],
 Class: gpe Features: ['backing', 'have', 'threaten', 'to', 'refer', 'to', 'the', 'which', 'could', 'impose'],
 Class: gpe Features: ['impose', 'sanction', 'if', 'it', 'find', 'have', 'violate', 'the', 'nuclear', 'treaty'],
 Class: art Features: ['find', 'tehran', 'have', 'violate', 'the', 'treaty', '<E5>', '<E4>', '<E3>', '<E

## Part 3 - Creating the table and splitting (10 points)

Here you're going to write the functions that create a data table with "document" vectors representing each instance and split the table into training and testing sets and random with an 80%/20% train/test split.

In [24]:
bigdf = a2.create_table(instances)
bigdf[20:40]

,class_label,0,1,2,3,4,5,6,7,8,...,290,291,292,293,294,295,296,297,298,299
20,gpe,1.306191,-1.393678,-0.856177,-0.159720,-0.232271,0.947830,-0.080729,0.316555,-0.557750,...,0.055124,-0.057452,0.039956,-0.050367,0.025615,0.013605,0.008567,-0.014485,0.019196,0.034361
21,geo,1.252120,1.073757,-0.137493,-0.622313,-0.065853,-0.218711,-0.054662,-0.134304,0.142867,...,-0.032305,0.031440,0.004685,0.011848,0.012704,0.007718,0.018015,-0.051808,-0.002584,-0.014393
22,gpe,1.406369,-1.615540,-0.943625,-0.023773,0.515520,-0.170753,-0.344187,-0.019561,-0.494608,...,0.027847,0.004815,0.019572,0.021553,0.022996,-0.015974,-0.038568,0.040157,0.001114,-0.029251
23,tim,0.799629,0.139391,0.846929,-0.336581,-0.159508,0.683823,-0.142515,0.179903,-0.180666,...,0.020929,-0.059583,0.025770,-0.065804,-0.054863,0.005939,0.138043,-0.058453,0.093844,0.019335
24,org,0.862888,0.407155,0.419224,-0.405613,-0.088230,-0.212306,-0.047591,-0.100860,0.093663,...,-0.015705,-0.016415,-0.005673,-0.048462,-0.044660,-0.008889,0.091033,-0.039493,0.058408,-0.017007
25,org,1.817505,-1.316782,-0.155113,-0.337877,0.541957,-0.388346,-0.391349,-0.155940,-0.679697,...,0.019466,0.029130,0.016828,-0.021648,0.011764,-0.017182,0.018476,-0.035019,0.003168,-0.020222
26,gpe,0.953128,0.013136,0.790544,-0.112943,1.455551,-0.579019,-0.726399,-0.199130,-0.944674,...,-0.035715,-0.028130,0.031168,-0.049351,0.039510,-0.024351,0.054437,-0.047914,-0.010928,-0.055949
27,gpe,0.692918,-0.044177,0.636781,-0.319148,0.005390,-0.209232,-0.062210,-0.090125,0.033261,...,0.048008,0.035037,0.025748,0.040363,0.118607,0.010036,0.016884,0.014214,0.050255,-0.056933
28,art,1.475658,1.262904,-0.397105,-0.695031,-0.008873,-0.270761,-0.071104,-0.141391,0.144866,...,0.080585,0.015673,0.048744,0.064552,0.034941,-0.097972,0.130679,-0.004468,0.065030,0.041262
29,gpe,1.316424,-1.639753,-0.995663,-0.101843,-0.185616,0.006845,-0.004194,0.018092,-0.035866,...,0.024070,-0.002715,0.010838,-0.005725,-0.002342,0.004583,-0.002098,-0.006686,0.003740,-0.011666


In [25]:
train_X, train_y, test_X, test_y = a2.ttsplit(bigdf)

# X and y mean feature matrix and class respectively.
train_X, train_y, test_X, test_y

(array([[ 8.94145766e-01,  2.09369861e-01,  7.67221426e-01, ...,
         -1.27111092e-02, -4.08251617e-02, -2.71728125e-03],
        [ 1.55102608e+00,  1.38428112e+00, -3.01299324e-01, ...,
         -7.45544066e-02, -1.77918957e-01,  1.24596480e-01],
        [ 9.60690805e-01, -1.14172530e+00, -6.77168161e-01, ...,
          3.58927582e-02, -4.28474998e-02, -4.18088752e-02],
        ...,
        [ 8.17490208e-01, -1.77385010e-02,  7.13453196e-01, ...,
          1.10160626e-01,  2.87199703e-02,  5.21909207e-02],
        [ 9.39612184e-01,  1.73633385e-01,  9.10231199e-01, ...,
         -3.84916556e-03,  6.71883419e-02, -1.88316748e-02],
        [ 2.69650912e+00,  2.05842898e-01,  2.79936887e+00, ...,
         -1.83845471e-02, -1.12738959e-02,  8.43093432e-03]]),
 4659    geo
 620     gpe
 4841    gpe
 716     gpe
 3335    geo
        ... 
 494     geo
 23      tim
 3925    gpe
 2970    org
 4777    geo
 Name: class_label, Length: 5022, dtype: object,
 array([[ 1.48020062e+00,  1.66771471

In [26]:
len(test_y) / (len(test_y) + len(train_y))

0.20006371455877667

In [27]:
len(test_X) / (len(test_X) + len(train_X))

0.20006371455877667

In [28]:
test_y.iloc[0]

'geo'

## Part 4 - Training the model (0 points)

This part you won't do yourself.

In [29]:
model = LinearSVC()
model.fit(train_X, train_y)
train_predictions = model.predict(train_X)
test_predictions = model.predict(test_X)

In [30]:
train_predictions

array(['geo', 'gpe', 'gpe', ..., 'gpe', 'per', 'org'], dtype=object)

In [31]:
train_y

4659    geo
620     gpe
4841    gpe
716     gpe
3335    geo
       ... 
494     geo
23      tim
3925    gpe
2970    org
4777    geo
Name: class_label, Length: 5022, dtype: object

In [32]:
test_predictions

array(['geo', 'geo', 'org', ..., 'geo', 'geo', 'per'], dtype=object)

In [33]:
test_y

4       geo
17      geo
25      org
26      gpe
33      gpe
       ... 
6246    org
6247    org
6263    geo
6266    geo
6273    per
Name: class_label, Length: 1256, dtype: object

## Part 5 - Evaluation (5 points)

Investigate for yourself what a "confusion matrix".  Then implement a function that takes the data and produces a confusion matrix in any readable form that allows us to compare the performance of the model by class.  

In [34]:
a2.confusion_matrix(test_y, test_predictions)

,gpe,art,eve,tim,org,per,nat,geo
gpe,1,0,3,0,0,1,1,2
art,0,1,5,0,0,1,1,3
eve,1,1,230,36,2,32,23,42
tim,0,0,57,89,0,19,35,24
org,0,0,0,0,0,0,0,1
per,0,0,71,42,0,68,22,28
nat,0,1,44,30,1,25,57,17
geo,0,1,90,22,0,24,9,93


In [35]:
a2.confusion_matrix(train_y, train_predictions)

,gpe,eve,art,tim,org,per,nat,geo
gpe,20,0,7,3,0,5,2,4
eve,0,14,13,2,0,0,0,4
art,1,1,1087,139,0,108,69,138
tim,2,0,206,482,0,93,93,64
org,0,0,3,1,12,0,0,2
per,0,1,213,123,0,349,80,71
nat,0,0,148,94,0,86,329,51
geo,0,0,265,86,0,76,48,427


Examine the matrix and describe your observations in README.md.  In particular, what do you notice about the predictions on the training data compared to those on the test data.

## Bonus Part A - Error analysis (2 points)

Look at the weakest-performing classes in the confusion matrix (or any, if they all perform poorly to the same extent).  Find some examples in the test data on which the classifier classified incorrectly for those classes.  What do you think is the reason why those are hard?  Consider linguistic factors and statistical factors, if applicable.  Write your answer in README.md.

## Bonus Part B - Expanding the feature space (7 points)

Run the entire process above, but incorporate part-of-speech tag information into the feature vectors.  It's your choice as to how to do this, but document it in README.md.  Your new process should run from the single call below:

In [36]:
a2.bonusb('/scratch/lt2222-v21-resources/GMB_dataset.txt')